In [1]:
# 新しく書き直し

# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.controls import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *
from optic.visualization import *

# アプリ名の設定
APP_NAME = "SUITE2P_ROI_CHECK"

class Suite2pROICheckGUI(QMainWindow):
    def __init__(self):
        """
        pri: 格納するkey
        """
        QMainWindow.__init__(self)
        self.widget_manager = WidgetManager()
        self.data_manager = DataManager()
        self.control_manager = ControlManager()
        self.config_manager = ConfigManager()
        self.config_manager.setCurrentApp(APP_NAME)

        self.setupUI_done = False
        setupMainWindow(self, self.config_manager.gui_defaults)
        
        # self.initializeAttributes()

        # self.setupFileLoadUI()

    """
    setup UI Function
    """
    def setupFileLoadUI(self):
        self.mainLayout = self.makeLayoutMain()
        self.mainLayout.addLayout(self.makeLayoutSectionBottom(), 1, 0, 1, 3)
        
        # bindFunc
        self.bindFuncFileLoadUI()

    # setup
    def setupUI(self):
        app_keys = self.gui_defaults["APP_KEYS"]
        # load data
        self.data_manager.dict_Fall[app_keys] = loadFallMAT(q_window=self,
                                                            data_manager=self.data_manager,
                                                            key_dict_Fall=app_keys,
                                                            path_fall=self.widget_manager.dict_lineedit[f"{app_keys}_path_fall"].text())

        if self.data_manager.dict_Fall[app_keys]:
            # make layout
            self.mainLayout.addLayout(self.makeLayoutSectionLeftUpper(), 0, 0, 1, 1)
            self.mainLayout.addLayout(self.makeLayoutSectionMiddleUpper(), 0, 1, 1, 1)
            self.mainLayout.addLayout(self.makeLayoutSectionRightUpper(), 0, 2, 1, 1)
            # setup
            self.widget_manager.dict_table[app_keys] = setupWidgetROITable(q_table=self.widget_manager.dict_table[app_keys], 
                                                                            len_row=len(self.data_manager.dict_Fall[app_keys]["F"]), 
                                                                            dict_tablecol=self.dict_tablecol)

            # bindFunc
            self.bindFuncAllWidget()

            # controls class
            self.control_manager.table_controls[app_keys] = TableControls(q_table=self.widget_manager.dict_table[app_keys],
                                                                          dict_tablecol=self.dict_tablecol,
                                                                          key_function_map=self.key_function_map)
            self.widget_manager.dict_table[app_keys].keyPressEvent = lambda event: self.control_manager.table_controls[app_keys].keyPressEvent(event)

    """
    makeLayout Function; Component
    小要素のLayout
    return -> Layout
    """
    # Main Layout
    def makeLayoutMain(self):
        central_widget = QWidget()
        self.setCentralWidget(central_widget)
        return QGridLayout(central_widget)

    "Bottom"
    # ファイル読み込み用UI Layout
    def makeLayoutComponentFileLoadUI(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        # LineEdit
        list_label = ["Fall mat file path", "Reference Tiff image file path (optional)", "Cellpose Mask path (optional)"]
        list_key = [f"{app_keys}_path_fall", f"{app_keys}_path_reftif", f"{app_keys}_path_cellpose"]
        for label, key in zip(list_label, list_key):
            layout.addLayout(makeLayoutLoadFileWidget(self.widget_manager, label=label, key_label=key, key_lineedit=key, key_button=key))
        # Button
        layout.addLayout(makeLayoutLoadFileExitHelp(self.widget_manager))
        return layout

    "Left Upper"
    # EventFileの読み込み, plot用
    def makeLayoutComponentEventFilePlot(self):
        layout = QVBoxLayout()
        # Eventのplot range
        layout.addLayout(makeLayoutLineEditLabel(self.widget_manager,
                                                 key_label="eventfile_align_plot_range",
                                                 key_lineedit="eventfile_align_plot_range",
                                                 label="plot range from Event start (pre, post; sec)",
                                                 text_set="(10, 10)"))
        # プロットに重ねるEventFile npyファイルの読み込みボタン, Clearボタン
        layout.addWidget(self.widget_manager.makeWidgetButton(key="loadEventFile", label="Load EventFile npy file"))  ### 要修正
        layout.addWidget(self.widget_manager.makeWidgetButton(key="clearEventFile", label="Clear"))
        return layout

    # trace plotの表示範囲
    def makeLayoutComponentTracePlotRange(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutLineEditLabel(self.widget_manager,
                                                key_label="minPlotRange",
                                                key_lineedit="minPlotRange",
                                                label="Minimum plot range (sec)",
                                                text_set="30"))
        layout.addWidget(self.widget_manager.makeWidgetCheckBox(key="plot_eventfile_trace", 
                                                 label="plot EventFile trace", 
                                                 checked=True,
                                                 ))
        return layout

    # 上記二つを合体
    def makeLayoutComponentEventFilePlot_TracePlotRange(self):
        layout = QHBoxLayout()
        layout.addLayout(self.makeLayoutComponentEventFilePlot())
        layout.addLayout(self.makeLayoutComponentTracePlotRange())
        return layout

    "Middle Upper"
    # ROI canvas
    def makeLayoutComponentROIView(self):
        layout = QVBoxLayout()
        layout.addWidget(self.widget_manager.makeWidgetView(key=self.gui_defaults["APP_KEYS"]))
        return layout

    # ROI property label, threshold lineedit
    def makeLayoutComponentROIPropertyDisplay_Threshold(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        layout.addLayout(makeLayoutROIProperty(self.widget_manager, key_label=f"{app_keys}_roi_prop"))
        layout.addLayout(makeLayoutROIThresholds(
            self.widget_manager, 
            key_label=f"{app_keys}_roi_threshold", 
            key_lineedit=f"{app_keys}_roi_threshold", 
            key_checkbox=f"{app_keys}_roi_threshold", 
            label_checkbox="ROI Show Threshold", 
            list_threshold_param=["npix", "compact"]))
        return layout

    # ROI display, background image button group, checkbox
    def makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutROITypeDisplay(self, self.widget_manager, key_buttongroup=f'{self.gui_defaults["APP_KEYS"]}_roi_type', dict_tablecol=self.dict_tablecol))
        layout.addLayout(makeLayoutBGImageTypeDisplay(self, self.widget_manager, key_buttongroup=f'{self.gui_defaults["APP_KEYS"]}_bgimage_type'))
        layout.addLayout(makeLayoutROIChooseSkip(self.widget_manager, key_checkbox=f'{self.gui_defaults["APP_KEYS"]}_roi_skip_choose', dict_tablecol=self.dict_tablecol))
        return layout

    # channel contrast, ROI opacity slider
    def makeLayoutComponentContrastOpacitySlider(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        channels = self.gui_defaults["CHANNELS"]
        
        layout_channel = QHBoxLayout()
        for channel in channels:
            layout_channel.addLayout(makeLayoutContrastSlider(self.widget_manager, 
                                                            key_label=f"{app_keys}_{channel}", 
                                                            key_checkbox=f"{app_keys}_{channel}", 
                                                            key_slider=f"{app_keys}_{channel}", 
                                                            label_checkbox=f"Show {channel} channel", 
                                                            label_label=f"{channel} Value", 
                                                            checked=True))

        layout.addLayout(layout_channel)
        layout.addLayout(makeLayoutOpacitySlider(self.widget_manager, 
                                                key_label=app_keys, 
                                                key_slider=app_keys, 
                                                label=app_keys
                                                ))
        return layout

    "Right Upper"
    # Table, ROI count label, Set ROI Celltype, ROICheck IO
    def makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO(self):
        layout = QVBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        layout.addLayout(makeLayoutTableROICountLabel(self.widget_manager, key_label=app_keys, key_table=app_keys, dict_tablecol=self.dict_tablecol))
        layout.addLayout(makeLayoutAllROISetSameCelltype(self.widget_manager, key_button=app_keys, dict_tablecol=self.dict_tablecol))
        layout.addLayout(makeLayoutROICheckIO(self.widget_manager, key_button=app_keys))
        return layout

    # ROI Filter, threshold
    def makeLayoutComponentROIFilter(self):
        layout = QHBoxLayout()
        app_keys = self.gui_defaults["APP_KEYS"]
        layout.addLayout(makeLayoutROIFilterThreshold(self.widget_manager, key_label=app_keys, key_lineedit=app_keys, dict_roi_threshold=self.gui_defaults["ROI_THRESHOLDS"]))
        layout.addLayout(makeLayoutROIFilterButton(self.widget_manager, key_label=app_keys, key_button=app_keys))
        return layout
    

    """
    makeLayout Function; Section
    領域レベルの大Layout
    """
    # 左上
    def makeLayoutSectionLeftUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(makeLayoutCanvasTracePlot(self.widget_manager, 
                                                   key_figure=self.gui_defaults["APP_KEYS"], 
                                                   key_canvas=self.gui_defaults["APP_KEYS"], 
                                                   key_app=self.gui_defaults["APP_KEYS"]), 
                        stretch=1)
        layout.addLayout(self.makeLayoutComponentEventFilePlot_TracePlotRange())
        return layout

    # 中上
    def makeLayoutSectionMiddleUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentROIView())
        layout.addLayout(self.makeLayoutComponentROIPropertyDisplay_Threshold())
        layout.addLayout(self.makeLayoutComponentROIDisplay_BGImageDisplay_ROISkip())
        layout.addLayout(self.makeLayoutComponentContrastOpacitySlider())
        return layout
    
    # 右上
    def makeLayoutSectionRightUpper(self):
        layout = QVBoxLayout()
        layout.addLayout(self.makeLayoutComponentTable_ROICountLabel_ROISetSameCelltype_ROICheckIO())
        layout.addLayout(self.makeLayoutComponentROIFilter())
        return layout

    # 下
    def makeLayoutSectionBottom(self):
        layout = self.makeLayoutComponentFileLoadUI()
        return layout

    """
    bindFunc Function
    配置したwidgetに関数を紐づけ
    """
    def bindFuncFileLoadUI(self):        
        app_keys = self.gui_defaults["APP_KEYS"]
        list_key = [f"{app_keys}_path_fall", f"{app_keys}_path_reftif", f"{app_keys}_path_cellpose"]
        list_filetype = ["mat", "tiff", "npy"]
        for key, filetype in zip(list_key, list_filetype):
            bindFuncLoadFileWidget(q_widget=self, q_button=self.widget_manager.dict_button[key], q_lineedit=self.widget_manager.dict_lineedit[key], filetype=filetype)

        self.widget_manager.dict_button["load_file"].clicked.connect(lambda: self.setupUI())
        bindFuncExit(q_window=self, q_button=self.widget_manager.dict_button["exit"])

    def bindFuncAllWidget(self):
        app_keys = self.gui_defaults["APP_KEYS"]

        # ROICheck save load
        bindFuncROICheckIO(q_window=self, 
                        q_lineedit=self.widget_manager.dict_lineedit[f"{app_keys}_path_fall"], 
                        q_table=self.widget_manager.dict_table[f"{app_keys}"], 
                        q_button_save=self.widget_manager.dict_button[f"{app_keys}_save_roicheck"], 
                        q_button_load=self.widget_manager.dict_button[f"{app_keys}_load_roicheck"], 
                        dict_tablecol=self.dict_tablecol)


if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

KeyError: 'APP_KEYS'

{'ROI_OPACITY': 50,
 'SELECTED_ROI_OPACITY': 255,
 'MIN_PLOT_RANGE': 30,
 'INIT_POSITION_X': 100,
 'INIT_POSITION_Y': 100,
 'WIDTH': 1200,
 'HEIGHT': 200,
 'TITLE': 'Suite2pROICheckGUI',
 'APP_KEYS': 'pri',
 'CHANNELS': ['Green', 'Red'],
 'ROI_THRESHOLDS': {'npix': '(50, 200)',
  'radius': '(3, 12)',
  'aspect_ratio': '(0, 1.5)',
  'compact': '(0, 1.5)',
  'skew': '(1, 100)',
  'std': '(0, 100)'}}

In [7]:
!pip install opencv-python

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ----------- ---------------------------- 11.0/38.8 MB 52.9 MB/s eta 0:00:01
   --------------------------------- ------ 32.5/38.8 MB 79.4 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 77.2 MB/s eta 0:00:00
